In [73]:
!pip install pandas sqlalchemy psycopg2 openpyxl

In [74]:
import pandas as pd
import re
from sqlalchemy import create_engine
import os


In [75]:
file_path = "Combined_IFSC1.csv"

# Since it's a CSV file, use read_csv instead of read_excel
df = pd.read_csv(file_path)

df

,BANK,IFSC,BRANCH,ADDRESS,CITY1,CITY2,STATE,STD CODE,PHONE
0,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065001,RTGS-HO,"ABHYUDAYA BUILDING, KAMAL NATH MARG,NEHRU NAGA...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,0.0,9.653261e+09
1,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065002,ABHYUDAYA NAGAR,"ABHYUDAYA EDUCATION SOCIETY, OPP. BLDG. NO. 18...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.470264e+07
2,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065003,BAIL BAZAR,"KMSPMS SCHOOL, WADIA ESTATE, BAIL BAZAR-KURLA(...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.503220e+07
3,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065004,BHANDUP,"CHETNA APARTMENTS, J.M.ROAD, BHANDUP, MUMBAI-4...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.596316e+07
4,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065005,DARUKHANA,"POTIA IND.ESTATE, REAY ROAD (E), DARUKHANA, MU...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.377816e+07
...,...,...,...,...,...,...,...,...,...
176995,ZILA SAHAKRI BANK LIMITED GHAZIABAD,ZSBL0000301,BHOJPUR,BHOJPUR MODINAGAR,MODINAGAR,GHAZIABAD,UTTAR PRADESH,0.0,7.217895e+09
176996,ZILA SAHAKRI BANK LIMITED GHAZIABAD,ZSBL0000311,VIJAYNAGAR,R 6 ADVOCATE COLONY PRATAP VIHAR VIJAYNAGAR,GHAZIABAD,GHAZIABAD,UTTAR PRADESH,0.0,7.217895e+09
176997,ZILA SAHAKRI BANK LIMITED GHAZIABAD,ZSBL0000321,KHORA,KHODA COLONY KHODA,GHAZIABAD,GHAZIABAD,UTTAR PRADESH,0.0,7.217895e+09
176998,ZILA SAHAKRI BANK LIMITED GHAZIABAD,ZSBL0000331,RAVALI KALA,"VILL.RAVLIKALA,MURAD NAGAR",RAVLI,GHAZIABAD,UTTAR PRADESH,0.0,7.217895e+09


In [76]:
df.columns = df.columns.str.strip().str.lower()

# Rename to match schema
df = df.rename(columns={
    "bank": "bank",
    "ifsc": "ifsc",
    "branch": "branch",
    "address": "address",
    "city1": "city1",
    "city2": "city2",
    "state": "state",
    "std code": "stdCode",
    "phone": "phone"
})
for col in ["bank", "branch", "address", "city1", "city2", "state"]:
    df[col] = df[col].astype(str).str.strip()

df.head()


,bank,ifsc,branch,address,city1,city2,state,stdCode,phone
0,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065001,RTGS-HO,"ABHYUDAYA BUILDING, KAMAL NATH MARG,NEHRU NAGA...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,0.0,9.653261e+09
1,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065002,ABHYUDAYA NAGAR,"ABHYUDAYA EDUCATION SOCIETY, OPP. BLDG. NO. 18...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.470264e+07
2,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065003,BAIL BAZAR,"KMSPMS SCHOOL, WADIA ESTATE, BAIL BAZAR-KURLA(...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.503220e+07
3,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065004,BHANDUP,"CHETNA APARTMENTS, J.M.ROAD, BHANDUP, MUMBAI-4...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.596316e+07
4,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065005,DARUKHANA,"POTIA IND.ESTATE, REAY ROAD (E), DARUKHANA, MU...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.377816e+07


In [77]:
import numpy as np
df['bank'] = df['bank'].str.strip()
df['bank'] = df['bank'].replace('', np.nan)
df

,bank,ifsc,branch,address,city1,city2,state,stdCode,phone
0,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065001,RTGS-HO,"ABHYUDAYA BUILDING, KAMAL NATH MARG,NEHRU NAGA...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,0.0,9.653261e+09
1,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065002,ABHYUDAYA NAGAR,"ABHYUDAYA EDUCATION SOCIETY, OPP. BLDG. NO. 18...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.470264e+07
2,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065003,BAIL BAZAR,"KMSPMS SCHOOL, WADIA ESTATE, BAIL BAZAR-KURLA(...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.503220e+07
3,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065004,BHANDUP,"CHETNA APARTMENTS, J.M.ROAD, BHANDUP, MUMBAI-4...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.596316e+07
4,ABHYUDAYA COOPERATIVE BANK LIMITED,ABHY0065005,DARUKHANA,"POTIA IND.ESTATE, REAY ROAD (E), DARUKHANA, MU...",MUMBAI,GREATER MUMBAI,MAHARASHTRA,22.0,2.377816e+07
...,...,...,...,...,...,...,...,...,...
176995,ZILA SAHAKRI BANK LIMITED GHAZIABAD,ZSBL0000301,BHOJPUR,BHOJPUR MODINAGAR,MODINAGAR,GHAZIABAD,UTTAR PRADESH,0.0,7.217895e+09
176996,ZILA SAHAKRI BANK LIMITED GHAZIABAD,ZSBL0000311,VIJAYNAGAR,R 6 ADVOCATE COLONY PRATAP VIHAR VIJAYNAGAR,GHAZIABAD,GHAZIABAD,UTTAR PRADESH,0.0,7.217895e+09
176997,ZILA SAHAKRI BANK LIMITED GHAZIABAD,ZSBL0000321,KHORA,KHODA COLONY KHODA,GHAZIABAD,GHAZIABAD,UTTAR PRADESH,0.0,7.217895e+09
176998,ZILA SAHAKRI BANK LIMITED GHAZIABAD,ZSBL0000331,RAVALI KALA,"VILL.RAVLIKALA,MURAD NAGAR",RAVLI,GHAZIABAD,UTTAR PRADESH,0.0,7.217895e+09


In [78]:
df.columns = df.columns.str.strip().str.upper()

# Extract unique banks
unique_banks = df["BANK"].dropna().unique()

# Create DataFrame with auto-generated IDs
bank_df = pd.DataFrame({
    "bank_id": range(1, len(unique_banks) + 1),
    "bank_name": unique_banks
})

print(bank_df)


     bank_id                                          bank_name
0          1                 ABHYUDAYA COOPERATIVE BANK LIMITED
1          2          THE ADARSH COOPERATIVE URBAN BANK LIMITED
2          3  THE AKOLA-WASHIM DISTRICT CENTRAL COOPERATIVE ...
3          4  THE AHMEDNAGAR DISTRICT CENTRAL COOPERATIVE BA...
4          5                       AIRTEL PAYMENTS BANK LIMITED
..       ...                                                ...
232      233                  VASAI VIKAS SAHAKARI BANK LIMITED
233      234             THE WEST BENGAL STATE COOPERATIVE BANK
234      235                                           YES BANK
235      236           THE ZOROASTRIAN COOPERATIVE BANK LIMITED
236      237                ZILA SAHAKRI BANK LIMITED GHAZIABAD

[237 rows x 2 columns]


In [79]:
# Reset index and drop the old one
bank_df = bank_df.reset_index(drop=True)

# Print without the DataFrame index
print(bank_df.to_string(index=False))

 bank_id                                                              bank_name
       1                                     ABHYUDAYA COOPERATIVE BANK LIMITED
       2                              THE ADARSH COOPERATIVE URBAN BANK LIMITED
       3         THE AKOLA-WASHIM DISTRICT CENTRAL COOPERATIVE BANK LTD., AKOLA
       4                   THE AHMEDNAGAR DISTRICT CENTRAL COOPERATIVE BANK LTD
       5                                           AIRTEL PAYMENTS BANK LIMITED
       6                                   THE AJARA URBAN CO OP BANK LTD AJARA
       7                              AMBARNATH JAIHIND COOP BANK LTD AMBARNATH
       8                               AKOLA JANATA COMMERCIAL COOPERATIVE BANK
       9                                  AHMEDABAD MERCANTILE COOPERATIVE BANK
      10                                     AHMEDNAGAR MERCHANTS COOP BANK LTD
      11                        AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED
      12                      THE ANDHRA

In [80]:
df.ifsc.unique()

AttributeError: 'DataFrame' object has no attribute 'ifsc'

In [ ]:
df = df.drop_duplicates(subset=["ifsc"])
df = df[df["ifsc"].notna()]
pattern = re.compile(r"^[A-Z]{4}0[A-Z0-9]{6}$")
df = df[df["ifsc"].apply(lambda x: bool(pattern.match(str(x))))]
df.head()

In [ ]:
# Count number of accounts per bank
bank_counts = df['bank'].value_counts(dropna=False)  # dropna=False to include NaN banks
print(bank_counts)



In [ ]:
bank_counts = df.groupby('bank').size().reset_index(name='num_accounts')

# Bank with highest accounts
highest_bank = bank_counts.loc[bank_counts['num_accounts'].idxmax()]
lowest_bank = bank_counts.loc[bank_counts['num_accounts'].idxmin()]

print(f"Highest: {highest_bank['bank']} with {highest_bank['num_accounts']} accounts")
print(f"Lowest: {lowest_bank['bank']} with {lowest_bank['num_accounts']} accounts")



In [ ]:
# Count unique branches per bank
branch_counts = df.groupby('bank')['branch'].nunique().reset_index(name='num_branches')

print(branch_counts)


In [ ]:
# Count unique branches per bank
branch_counts = df.groupby('bank')['branch'].nunique().reset_index(name='num_branches')

# Bank with most branches
most_branches = branch_counts.loc[branch_counts['num_branches'].idxmax()]
# Bank with fewest branches
fewest_branches = branch_counts.loc[branch_counts['num_branches'].idxmin()]

print(f"Bank with most branches: {most_branches['bank']} ({most_branches['num_branches']} branches)")
print(f"Bank with fewest branches: {fewest_branches['bank']} ({fewest_branches['num_branches']} branches)")


In [ ]:
df['branch'] = df['branch'].str.strip()
df['branch'] = df['branch'].replace('', np.nan)
print(df['branch'])

In [ ]:
df['address'] = df['address'].str.strip()
df['address'] = df['address'].replace('', np.nan)
print(df['address'])

In [ ]:
df['city1'] = df['city1'].str.strip()
df['city1'] = df['city1'].replace('', np.nan)
print(df['city1'])

In [ ]:
df['city2'] = df['city2'].str.strip()
df['city2'] = df['city2'].replace('', np.nan)
print(df['city2'])

In [ ]:
df.state.unique()

In [ ]:
state_corrections = {
    # Common misspellings
    'MADHY PRADESH': 'MADHYA PRADESH',
    'UTTA PRADES': 'UTTAR PRADESH',
    'MADHYAPRADESH': 'MADHYA PRADESH',
    'ODHISA': 'ODISHA',
    'ORISSA': 'ODISHA',
    'CHATTISGARH': 'CHHATTISGARH',
    'CHHATISGARH': 'CHHATTISGARH',
    'CHHATISHGARH': 'CHHATTISGARH',
    'HARKHAND': 'JHARKHAND',
    'JH': 'JHARKHAND',
    'KA': 'KARNATAKA',
    'KARANATAKA': 'KARNATAKA',
    'TN': 'TAMIL NADU',
    'TAMILNADU': 'TAMIL NADU',
    'TELENGANA': 'TELANGANA',
    'ANDHRAPRADESH': 'ANDHRA PRADESH',
    'GUJRAT': 'GUJARAT',
    'WESTBENGAL': 'WEST BENGAL',
    'ANDAMANANDNICOBAR': 'ANDAMAN AND NICOBAR ISLANDS',
    'JAMMUANDKASHMIR': 'JAMMU AND KASHMIR',
    'HIMACHALPRADESH': 'HIMACHAL PRADESH',
    'UTTARANCHAL': 'UTTARAKHAND',
    'CHANDIGARH UT': 'CHANDIGARH',
    'DADARA AND NAGAR HAVELI AND DAMAN AND DIU': 'DADRA AND NAGAR HAVELI AND DAMAN AND DIU',
    'DAMAN': 'DAMAN AND DIU',
    # Abbreviations
    'MH': 'MAHARASHTRA',
    'AP': 'ANDHRA PRADESH',
    'HP': 'HIMACHAL PRADESH',
    'MP' : 'MADHYA PRADESH',
    'UP' : 'UTTAR PRADESH',
    
}

df['state'] = df['state'].str.strip() 
df['state'] = df['state'].replace(state_corrections)
df['state'] = df['state'].replace('', np.nan)
df.state



In [ ]:
df.state.unique()

In [ ]:
df.stdCode.unique()

In [ ]:
df

In [ ]:
df[df['stdCode'].isna() | (df['stdCode'].astype(str).str.strip() == "")]
df[~df['stdCode'].astype(str).str.isnumeric()]
df['stdCode'].astype(str).str.len().value_counts()
df.stdCode

In [ ]:
df.phone.unique()

In [ ]:
df[df['phone'].isna() | (df['phone'].astype(str).str.strip() == "")]
df[df['phone'].isin(['9999999999','1234567890','0000000000'])]
df[~df['phone'].astype(str).str.match(r'^[0-9]+$', na=False)]['phone']
df['phone'].value_counts()
df[df['phone'].astype(str).str.startswith(('6','7','8','9','1800'))]

df[df['phone'].isin(['9999999999','1234567890','0000000000'])]

df.phone

In [ ]:
df['phone'] = df['phone'].astype(str).str.strip()
df['stdCode'] = df['stdCode'].astype(str).str.strip()

df_with_match = df[df.apply(lambda row: row['phone'].startswith(row['stdCode']), axis=1)]
df.phone

In [ ]:
import math

# Clean the phone column
df['phone'] = df['phone'].apply(
    lambda x: str(int(float(x))) if pd.notna(x) and str(x).strip() not in ["", "nan"] else None
)
df.phone

In [ ]:
import numpy as np 
df['phone'] = df['phone'].replace({'': np.nan, 'nan': np.nan})
df.phone

In [ ]:
df['phone'] = df['phone'].replace({'': np.nan, 'nan': np.nan})
df.phone

In [ ]:
# 1. Create a city-to-STD mapping (expand as needed)
city_std_map = {
    "NEW DELHI": "011",     # Metro city
    "MUMBAI": "022",
    "KOLKATA": "033",
    "CHENNAI": "044",
    "PUNE": "020",
    "HYDERABAD": "040",
    "AHMEDABAD": "079",
    "BANGALORE": "080",
    # Tier-2 cities (Wikipedia)
    "GHAZIABAD": "120",
    "NOIDA": "120",
    "GURGAON": "124",
    "FARIDABAD": "129",
    "DEHRADUN": "135",
    "JAIPUR": "141",
    "LUDHIANA": "161",
    "PATIALA": "175",
    "JALANDHAR": "181",
    "AMRITSAR": "183",
    "SANGLI": "233",
    "AURANGABAD": "240",
    "AHMEDNAGAR": "241",
    "VASAIVIRAR": "250",
    "KALYAN": "251",
    "NASHIK": "253",
    "AGRA": "562",
    "SURAT": "261",
    "VADODARA": "265",
    "PUDUCHERRY": "413",
    "COIMBATORE": "422",
    "ERODE": "424",
    "MADURAI": "452",
    "THIRUVANANTHAPURAM": "471",
    "KOLLAM": "474",
    "KOCHI": "484",
    "BENGALURU": "080",
    "HUBLI": "836",
    "GUNTUR": "863",
    "VIJAYAWADA": "866",
    "WARANGAL": "870",
    "VISAKHAPATNAM": "891",
    "KOLHAPUR": "231",
    "MANGALAGIRI": "864",     # near Guntur/Vijayawada
    "NAGRAM": "522",          # Lucknow district
    "KAKORI": "522",          # Lucknow district
    "ITAUNJA": "522",         # Lucknow district
    "BARADGAON SUDRIK": "211",
    # Vizianagaram District
    "VIZIANAGARAM": "8922",
    "BOBBILI": "8944",
    "CHEEPURUPALLE": "8922",   # shares Vizianagaram
    "GAJAPATHINAGARAM": "8924",
    "PARVATHIPURAM": "8945",
    "SALUR": "8941",
    "SRUNGAVARAPUKOTA": "8921",
    "GARIVIDI": "8925",
    "KOTHAVALASA": "8920",
    
    # Visakhapatnam District
    "VISHAKHAPATNAM": "891",
    "ANAKAPALLE": "891",
    "BHEEMUNIPATNAM": "891",
    "NARSIPATNAM": "8932",
    "YELLAMANCHILI": "8933",
    "PADERU": "8936",

    # Krishna District
    "VIJAYAWADA": "866",
    "MACHILIPATNAM": "8672",
    "NUZVIDU": "8673",
    "GUDIVADA": "8674",
    "AVANIGADDA": "8671",
    
    # Guntur District
    "GUNTUR": "863",
    "TENALI": "8644",
    "NARASARAOPET": "8647",
    "VINUKONDA": "8646",
    "SATTENAPALLE": "8641",
    
    # Nellore District
    "NELLORE": "861",
    "GUDUR": "8624",
    "KAVALI": "8626",
    "SULLURPET": "8623",

    # Chittoor District
    "CHITTOOR": "8572",
    "TIRUPATI": "877",
    "MADANAPALLE": "8571",
    "PUTTUR": "8577",
    "NAGARI": "8577",

    # Anantapur District
    "ANANTAPUR": "8554",
    "DHARMAVARAM": "8559",
    "HINDUPUR": "8556",
    "GUNTAKAL": "8552",
    "RAYADURG": "8555",
    "TADPATRI": "8558",

    # Kadapa District
    "KADAPA": "8562",
    "PRODDATUR": "8564",
    "RAYACHOTY": "8561",
    "JAMMALAMADUGU": "8563"
}
df['city1'] = df['city1'].astype(str).str.upper().str.strip()
df['stdCode'] = df['city1'].map(city_std_map)
df.stdCode





In [ ]:
df.std

In [ ]:
df['stdCode'] = df['stdCode'].apply(
    lambda x: str(int(float(x))) if pd.notna(x) and str(x).strip() not in ["", "nan"] else None
)
df

In [ ]:
df.to_csv("cleaned_bankifsc_data.csv", index=False)